In [11]:
pd.set_option('display.max_rows', None)


In [12]:
pip install requests textblob


Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [17]:
# Import necessary libraries
import pandas as pd
import requests
from textblob import TextBlob

API_KEY = '9923aaa2a3b2777bfdeba7f76c97d212'

BASE_URL = 'https://api.themoviedb.org/3/search/movie'

file_path = 'movie.metadata.tsv'
movie_data = pd.read_csv(file_path, sep='\t', header=None)

movie_titles = movie_data[2].tolist()

def get_movie_summary(title):
    params = {
        'api_key': API_KEY,
        'query': title,
        'language': 'en-US'
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['overview']
        else:
            return None
    else:
        print(f"Error fetching data for {title}: {response.status_code}")
        return None

def analyze_sentiment(summary):
    if summary:
        analysis = TextBlob(summary)
        if analysis.sentiment.polarity > 0.5:
            return 5  # Very happy ending
        elif 0.1 < analysis.sentiment.polarity <= 0.5:
            return 4  # Happy ending
        elif -0.1 <= analysis.sentiment.polarity <= 0.1:
            return 3  # Neutral ending
        elif -0.5 < analysis.sentiment.polarity < -0.1:
            return 2  # Sad ending
        else:
            return 1  # Very sad ending
    else:
        return None

num_movies_to_process = 10000
movie_titles_sample = movie_titles[:num_movies_to_process]

results = []
for title in movie_titles_sample:
    summary = get_movie_summary(title)
    score = analyze_sentiment(summary)
    if summary: 
        results.append({'Title': title, 'Summary': summary, 'Score': score})

results_df = pd.DataFrame(results)

movie_data_cleaned = movie_data.copy()
movie_data_cleaned['Score'] = pd.Series(results_df['Score'])

output_file_path = 'movies_dataset_cleaned.tsv'
movie_data_cleaned.to_csv(output_file_path, sep='\t', index=False)

print(f"Cleaned dataset saved to {output_file_path}")


Cleaned dataset saved to movies_dataset_cleaned.tsv
